In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import seaborn as sns
import networkx as nx

In [7]:
# read in code csv
key = pd.read_csv('../data/code_key.csv')
seeds = key['Seed Value'].unique().tolist()
key.head()
# get unique list of seeds
seeds = sorted(seeds)
seed_dict = dict(zip(key['Seed Value'], key['Number of Incidents']))


# read in feeder csv
filename = '../data/feeder_links/feeders.csv'
feeders = pd.read_csv(filename)

In [9]:
# helper functions

def get_same_indices(link_vol_df, avg_delay_df):

    # Make sure link id's are unique
    vol_i, avg_del_i = set(link_vol_df.index), set(avg_delay_df.index)

    # Find the links that are unique to each DataFrame
    vol__i, avg_del__i = list(vol_i - avg_del_i), list(avg_del_i - vol_i)

    # Create a boolean masks to identify rows with Link IDs in avg_del__i_list
    mask = avg_delay_df.index.isin(avg_del__i)
    avg_delay_df = avg_delay_df[~mask]

    mask = link_vol_df.index.isin(vol__i)
    link_vol_df = link_vol_df[~mask]
    
    for j in range(len(link_vol_df)):
        if link_vol_df.index[j] != avg_delay_df.index[j]:
            print(f'ERROR: link_vol: {link_vol_df.index[j]}, avg_delay: {avg_delay_df.index[j]}')

    if len(link_vol_df) != len(avg_delay_df):
        print('oops!')
        return 0
    else:
        return link_vol_df, avg_delay_df

In [3]:
# test for seed 637

# get average delay
filepath ='../data/average_delay'
filename = f'{filepath}/Seed 637/1-19-637.delays_perLink_100.csv'
#filename = f'{filepath}/Seed 637/2-19-637.delays_perLink_100.csv'
avg_delay_637 = pd.read_csv(filename, index_col=0)


# get link volumes
filepath ='../data/link_volumes'
filename = f'{filepath}/Seed 637/1-19-637.volume.csv'
#filename = f'{filepath}/Seed 637/2-19-637.volume.csv'
link_vol_637 = pd.read_csv(filename, index_col=0)

# fix column issue
#link_vol_637.columns = avg_delay_637.columns.to_list()
#filename = '/Users/brynnwoolley/Library/CloudStorage/Box-Box/MATSim_Analysis/Brynn_files/Link Volumes/Baseline/0-0-000.volume.csv'
#link_vol_637.to_csv(filename)

# make sure they have the same indices & that they're in the same order
link_vol_637, avg_delay_637 = get_same_indices(link_vol_637, avg_delay_637)
for i in range(len(link_vol_637)):
    if link_vol_637.index[i] != avg_delay_637.index[i]:
        print(f'ERROR: link_vol: {link_vol_637.index[i]}, avg_delay: {avg_delay_637.index[i]}')

# fill na values with 0
#link_vol_637, avg_delay_637 = link_vol_637.fillna(0, inplace=True), avg_delay_637.fillna(0, inplace=True)

# compute delay?
delay_637 = avg_delay_637 * link_vol_637.astype(float)


NameError: name 'get_same_indices' is not defined

In [5]:
def test_baseline():
    """ Check if it seems to work, baseline edition"""

    # get average delay
    filepath ='../data/average_delay'
    filename = f'{filepath}/Baseline/0-0-000.baseline.delays_perLink_100.csv'
    base_avg_delay = pd.read_csv(filename, index_col=0)


    # get link volumes
    filepath ='../data/link_volumes'
    filename = f'{filepath}/Baseline/0-0-000.volume.csv'
    base_link_vol = pd.read_csv(filename, index_col=0)


    # make sure they have the same indices & that they're in the same order
    base_link_vol, base_avg_delay = get_same_indices(base_link_vol, base_avg_delay)
    for i in range(len(base_link_vol)):
        if base_link_vol.index[i] != base_avg_delay.index[i]:
            print(f'ERROR: link_vol: {base_link_vol.index[i]}, avg_delay: {base_avg_delay.index[i]}')

    # try to get delay
    base_delay = base_avg_delay * base_link_vol.astype(float)

    # get list of indices
    base_delay['Total'] = base_delay.sum(axis=1)
    n = 5
    df = base_delay.nlargest(n, 'Total')
    base_delay = base_delay.drop('Total', axis=1)

    indices = df.index.to_list()

    # get filtered df's
    filtered_base_delay = base_delay[base_delay.index.isin(indices)].T
    filtered_base_avg_delay = base_avg_delay[base_avg_delay.index.isin(indices)].T
    filtered_base_link_vol = base_link_vol[base_link_vol.index.isin(indices)].T

    for id in filtered_base_avg_delay.columns:
        plt.subplot(1, 2, 1)
        plt.plot(filtered_base_avg_delay.index, filtered_base_avg_delay[id], label='avg delay')
        plt.plot(filtered_base_link_vol.index, filtered_base_link_vol[id], label='link vol')
        plt.ylabel('Measures')
        plt.xlabel('Time')
        plt.title(f'Link ID: {id}')
        plt.legend()
        plt.tight_layout()
        plt.subplot(1, 2, 2)
        plt.plot(filtered_base_delay.index, filtered_base_delay[id], label='delay')

        plt.ylabel('Measures')
        plt.xlabel('Time')
        plt.title(f'Link ID: {id}')
        plt.legend()
        plt.tight_layout()
        plt.show()
#test_baseline()



### CODE TO SAVE BASELINE DELAY

In [4]:
def save_baseline_delay():
    # get average delay
        filepath ='../data/average_delay'
        filename = f'{filepath}/Baseline/0-0-000.baseline.delays_perLink_100.csv'
        base_avg_delay = pd.read_csv(filename, index_col=0)


        # get link volumes
        filepath ='../data/link_volumes'
        filename = f'{filepath}/Baseline/0-0-000.volume.csv'
        base_link_vol = pd.read_csv(filename, index_col=0)


        # make sure they have the same indices & that they're in the same order
        base_link_vol, base_avg_delay = get_same_indices(base_link_vol, base_avg_delay)
        for i in range(len(base_link_vol)):
            if base_link_vol.index[i] != base_avg_delay.index[i]:
                print(f'ERROR: link_vol: {base_link_vol.index[i]}, avg_delay: {base_avg_delay.index[i]}')

        # try to get delay
        base_delay = base_avg_delay * base_link_vol.astype(float)

        return base_delay

base_delay = save_baseline_delay()
filename = '../data/link_delays/Baseline/0-0-000.delay.csv'
base_delay.to_csv(filename)

### CODE TO SAVE ALL OTHER DELAY

In [10]:
for seed in seeds:
    # get number of incidents for that seed
    n = seed_dict[seed]

    # make 'Seed ###' folders within Delay Folder
    seed_folder = f'../data/link_delays/Seed {seed}'
    if not os.path.exists(seed_folder):
        os.mkdir(seed_folder)
    
    # iterate through i(mt_deployment) = 1, 2, 3
    for i in range(1,4):
        # first get average delay and link volume:
        filename1 = f'../data/average_delay/Seed {seed}/{i}-{n}-{seed}.delays_perLink_100.csv'
        avg_df = pd.read_csv(filename1, index_col=0)
        filename2 = f'../data/link_volumes/Seed {seed}/{i}-{n}-{seed}.volume.csv'
        vol_df = pd.read_csv(filename2, index_col=0)

        # fix the column naming issue
        vol_df.columns = avg_df.columns.to_list()
        #filename = f'Link Volumes/Seed {seed}/{i}-{n}-{seed}.volume.csv'
        #link_vol_637.to_csv(filename)


        # ensure they have the same indices:
        link_vol_df, avg_delay_df = get_same_indices(vol_df, avg_df)
        link_vol_df, avg_delay_df = link_vol_df.astype(float), avg_delay_df.astype(float)

        # compute delay
        delay = link_vol_df * avg_delay_df


        # save it as a properly labeled csv
        filename = f'../data/link_delays/Seed {seed}/{i}-{n}-{seed}.delay.csv'

        delay.to_csv(filename)


